# MEIRLOP Documentation
Motif Enrichment In Ranked Lists Of Peaks (MEIRLOP) analyzes the relative enrichment of transcription factor binding motifs found in genomic sequences at the top or bottom of a given ranking/score. It does this by calculating motif enrichment using a logistic regression model that describes the log-likelihood of a motif being present in sequences with higher/lower scores, while accounting for the effect of multiple covariates, such as sequence bias.

This document will show you how to quickly perform an analysis in MEIRLOP for those who just want to get started, followed by more detailed explanations about interpretation, usage, input file formats, and best practices. For a full demonstration of how we used MEIRLOP 'for real', see our [walkthrough](walkthrough.ipynb).

# Prerequisites
Make sure you have the following packages and their dependencies installed:
* MEIRLOP (obviously)
* Wget

If you don't, you can run the following command to install. You will have to confirm the installed packages:

```
conda install -c bioconda -c conda-forge -c npdeloss meirlop wget
```


# Quickstart Example

## Download input data (optional)
Skip to [Run MEIRLOP](#Run-MEIRLOP) if you have your own files.

In [1]:
%%bash
# URLs for downloaded files
SCORED_FASTA_FILE_URL="https://npdeloss.github.io/meirlop/notebooks/archived_data/dhs.h3k27ac_over_h3k27me3_mean.scored.fa"
COVARIATES_FILE_URL="https://npdeloss.github.io/meirlop/notebooks/archived_data/dhs.dnase.covariates.tsv"
JASPAR_MOTIFS_FILE_URL="http://jaspar2018.genereg.net/download/CORE/JASPAR2018_CORE_vertebrates_non-redundant_pfms_jaspar.txt"
# User agent string, for spoofing the user agent to download the motifs file
USER_AGENT="Mozilla/5.0 (X11; Linux i686 (x86_64)) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.75 Safari/537.36"
# Download scored FASTA file
wget -O scored_fasta.fa -o scored_fasta.fa.log "$SCORED_FASTA_FILE_URL"
# Download covariates file
wget -O covariates.tsv -o covariates.tsv.log "$COVARIATES_FILE_URL"
# Download JASPAR-formatted motifs file
wget -O motifs.txt -o motifs.txt.log -U "$USER_AGENT" "$JASPAR_MOTIFS_FILE_URL"


## Run MEIRLOP

In [2]:
%%bash

# Set number of threads. Adjust according to your computational environment.
NUM_THREADS=30
# Run MEIRLOP
OMP_NUM_THREADS="$NUM_THREADS" \
meirlop \
--jobs "$NUM_THREADS" \
--covariates covariates.tsv \
--html \
--sortabs \
--fa scored_fasta.fa \
motifs.txt \
meirlop_output_directory/ \
&> meirlop_output.log

# Delete this file, it's too big for Github
# OPTIONAL if not committing the output to Github
rm meirlop_output_directory/motif_peak_set_dict.json

# Viewing MEIRLOP Results
If you ran MEIRLOP with the `--html` flag, you have access to an [HTML report](meirlop_output_directory/lr_results.html) available in [`meirlop_output_directory/lr_results.html`](meirlop_output_directory/lr_results.html). We can show this in a Jupyter notebook using the display and IFrame functions, demonstrated below. The HTML output is useful because it embeds the motif logos and has quality of life features including color-coded bar plots to visualize values, search capabilities, and the ability to sort along arbitrary columns. To learn what those columns mean, see ["Interpreting MEIRLOP results"](#Interpreting-MEIRLOP-results).

If you opted not to use the `--html` flag, you are limited to the [tab-separated results file](meirlop_output_directory/lr_results.tsv) in [`meirlop_output_directory/lr_results.tsv`](meirlop_output_directory/lr_results.html).

In [3]:
from IPython.display import display, IFrame

In [4]:
display(IFrame('meirlop_output_directory/lr_results.html', 1600, 1550))

# Interpreting MEIRLOP results
MEIRLOP calculates an enrichment coefficient for each motif (`coef` in the results table) that is more positive for motifs that are more likely to appear in sequences with higher scores, and more negative for motifs that are more likely to appear in sequences with lower scores (after accounting for the effect of covariates). The magnitude of this coefficient is available in the `abs_coef` column, while the significance before and after multiple testing correction is reflected in the p-value (`pval`) and adjusted p-value (`padj`) columns. The column `padj_sig` is set to 1 for motifs that pass a significance thresold (default 0.05, set with `--padj`) after multiple testing, and to 0 for other motifs. 

By default, motifs are sorted such that motifs that pass the threshold are at the top of the results, with significant results ordered from highest enrichment coefficient to lowest (Sort by `padj_sig` then `coef`, in that priority). However, if you are interested in motifs likely to appear at either extreme of the sequence scores, you can run MEIRLOP with the `--sortabs` flag to order significant results from the highest absolute enrichment coefficient to lowest absolute enrichment coefficent (Sort by `padj_sig` then `abs_coef`, in that priority). 

For technical details of how MEIRLOP arrives at enrichment coefficients, see ["How MEIRLOP works"](#How-MEIRLOP-works).

# Usage and command line parameters
MEIRLOP has many command line parameters, and a few useful extras we recommend using when calling it.
```
usage: meirlop [-h] --fa scored_fasta_file
               [--jobs jobs] [--scan] [--html]
               [--svg] [--sortabs] [--norevcomp] [--kmer max_k] [--length]
               [--gc] [--covariates covariates_table_file]
               [--score score_column] [--pval scan_pval_threshold]
               [--pcount scan_pseudocount]
               [--padj logistic_regression_padj_threshold]
               motif_matrix_file output_dir
```

Here we explain the most common elements of a MEIRLOP command line invocation:
* `NUM_THREADS=30`: The number of threads. We use 30 threads. Adjust to suit your computational environment
* `OMP_NUM_THREADS="$NUM_THREADS"`: Specify this or else statsmodels hogs all the cores for logistic regression (**Optional**, default: statsmodels hogs all cores).
* `$(which time) --verbose`: Track resource usage using the `time` command. (**Optional** but recommended)
    * Omitted from the quickstart for simplicity.
    * Requires you install the time command (e.g. via `conda install -c conda-forge time`)
* `meirlop`: You're running MEIRLOP. (**MANDATORY**)
* `--jobs "$NUM_THREADS"`: Enable multithreading on the number of threads specified (**Optional**, default: no multithreading).
* `--kmer 2`: Control for sequence bias, quantified through **di**nucleotide frequencies. (**Optional**, default: same behavior as described in this command)
    * Omitted from the quickstart for simplicity.
* `--covariates covariates.tsv`: Control for the experimental covariates in the specified file. (**Optional**, default: Don't use covariates other than sequence bias)
    * * For details on what this should look like, see ["Covariates TSV files"](#Covariates-TSV-files)
* `--pcount 0.001 --pval 0.001`: Set the pseudocount and p-value for the underlying MOODS motif scanner. (**Optional**, default: 0.001 for both values)
    * In [AME](http://meme-suite.org/doc/ame.html?man_type=web), the argument `--motif-pseudo` is analogous to `--pcount` in MEIRLOP, and defaults to 0.1. When comparing to [AME](http://meme-suite.org/doc/ame.html?man_type=web), you may want to adjust these values.
    * Omitted from the quickstart for simplicity.
* `--padj 0.05`: Set the adjusted p-value threshold for logistic regression results. (**Optional**, default: set threshold to 0.05)
    * Omitted from the quickstart for simplicity. Rarely used.
* `--html`: Enable pretty HTML output (**Optional** but highly recommended). 
    * If you need prettier figures at the cost of a clunkier browser experience, you can also enable the `--svg` option, which doesn't pixelate motif logos when resized.
* `--scan`: Output data about where motifs were detected in the input sequences. Takes a while to write, but can come in handy (**Optional**, default: Don't output motif scan data).
    * Omitted from the quickstart for simplicity.
* `--sortabs`: Sort by the absolute value of the enrichment coefficient. (**Optional**, default: Sort by motif enrichment coefficient, higher values first)
    * If you're interested in motifs enriched towards either extreme of a score, this option is for you.
    * Other packages like [HOMER](http://homer.ucsd.edu/homer/introduction/basics.html) and [AME](http://meme-suite.org/doc/ame.html?man_type=web) assume a one-sided analysis, and would require two runs to get a picture of both extremes of the score.
* `--fa scored_fasta.fa`: Analyze our scored fasta file that we generated (Analysis of a scored fasta file is **MANDATORY**).
    * For details on what this should look like, see ["Scored FASTA files"](#Scored-FASTA-files)
* `motifs.txt` Use the JASPAR 2018 motif database we just downloaded (using a motif database is **MANDATORY**).
    * You can use other motif databases as long as they are in JASPAR format.
    * For details on what this should look like, see ["Motifs JASPAR files"](#Motifs-JASPAR-files)
* `meirlop_output_directory`: Output will be in `meirlop_output_directory` (**MANDATORY**)...
* `&> meirlop_output.log`: ... with a similarly named log file (**Optional** but recommended).

Other parameters are explained in the README and when calling `meirlop -h`, but these cover 95% of use cases.

# Input File formats
MEIRLOP uses standard file formats for input. When preparing files for your own analysis, follow these examples.

## Scored FASTA files
For input of scored sequences, MEIRLOP accepts a scored FASTA file in the [AME](http://meme-suite.org/doc/ame.html?man_type=web) style. That's like a normal FASTA file, but the sequence header has a space separating the sequence name and the sequence score. In addition, sequence names must be unique.

MEIRLOP prefers that these sequences be of the same length, although we have experimental features to relax this constraint.

See the `head` output below for an example.

In [5]:
%%bash
head scored_fasta.fa

>chr1:135117 0.04540440142154693
GCTGGGCCTAAAGAGGCCACTGGCAGGCAGGAGCTGGACCTGCCGAAGTGGCCGAAAGGCAGGAGCTTTGGACTGGGGAGGCCGCAGTGAGGCGAGAGCTAGCTGGGCGTGGAGAGTCCGCTGTGAGGCCGAGGCCGAGGCTGGGCCCGT
>chr1:139386 0.1429919958114624
AAGCTGGGCGTGAAGAATCTGCTGTGAGGCAGACGTTGGGACTGTAGAGGCTGACGGGAGGCAGAGGCTGGGCCTGGAGGGGCCACCAAGATGCAGGAGCTGGGCCTGGAGAGGCTGCAAAGAAGCATGAGCTGGGCCTGGTGAGGTCAA
>chr1:180826 0.0
TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACAACCCTAACCCTAACCCTAACAACCCTAACCCTAACCCTAACCCTAACCCTAAC
>chr1:181490 0.0
CGTGGCGCAGGCGCAGAGAGGCGCGCCGTGCTGCCGCAGGCGCAGAGACACATGCTAGCGCGTCCAGGGGGTGGAGGCGTGGCGCAGGCGCAGAGACGCACGCCTACGGGCGGGGTTGGGGGGGGCGTGTGTTACAGGAGCAAAGTCGCA
>chr1:268065 0.02270220071077347
TTCTCTGTGCAGCACCAGGTGGCAGCAGAGGTCAGCAAGGCAAACCCGAGCCCGGGGATGCGGAGTGGGGGCAGCTACGTCCTCTCTTGAGCTACAGCAGATTCACTCTGTTCTGTTTCATTGTTGTTTAGTTTGCGTTGTGTTTCTCCA


## Covariates TSV files
MEIRLOP also takes a covariates file that it can use alongside the ones it calculates for sequence bias. This is simply a tab-separated values file with two (or more) labeled columns, in order:
1. The name of the sequence (`peak_id`)
2. Any additional covariate values

The sequence names must match those in the scored FASTA file.

See the `head` output below for an example.

In [6]:
%%bash
head covariates.tsv

peak_id	dnase
chr1:135117	17.693333333333328
chr1:139386	18.773333333333333
chr1:180826	32.89333333333333
chr1:181490	203.2333333333333
chr1:268065	208.7333333333333
chr1:268500	8.666666666666666
chr1:608551	4.733333333333333
chr1:629230	125.7
chr1:629554	81.61333333333333


## Motifs JASPAR files
MEIRLOP finds the enrichment of known motifs (for now), so we must supply a motif matrix file in JASPAR format. You can usually download these from [JASPAR](http://jaspar.genereg.net/downloads/), although other motif databases may also have their motifs in this format.

See the `head` output below for an example. You'll need to download the file first: we're not going to keep a copy of JASPAR's motifs in our Github repository.

In [7]:
%%bash
head motifs.txt

>MA0004.1	Arnt
A  [     4     19      0      0      0      0 ]
C  [    16      0     20      0      0      0 ]
G  [     0      1      0     20      0     20 ]
T  [     0      0      0      0     20      0 ]
>MA0006.1	Ahr::Arnt
A  [     3      0      0      0      0      0 ]
C  [     8      0     23      0      0      0 ]
G  [     2     23      0     23      0     24 ]
T  [    11      1      1      1     24      0 ]


# Best Practices
When using MEIRLOP, there are a few things to keep in mind.

## Selecting a motifs JASPAR file
In most of our work we use the [JASPAR 2018 CORE vertebrate non-redundant motif set](http://jaspar2018.genereg.net/downloads/), which consists of 579 motifs. We selected this motif set to limit redundancy between different versions of what are essentially the same motif (i.e. avoid many motifs matching the same transcription factor), and to restrict the motifs tested to those relevant for analyzing human data. Both of these considerations affect results when MEIRLOP performs multiple testing correction for enrichment hypotheses of multiple motifs.

We recommend that you follow similar principles when selecting your motifs file: Avoid redundant motif entries, and restrict the motifs only to those relevant for the species under analysis.

## Choosing k-mer length
By default, MEIRLOP controls for dinucleotide frequencies when performing its motif enrichment. While it is possible to control for longer kmers, consider the length of the kmer relative to the length of the motifs in your motifs file: Correcting for kmers of length 7 with motifs of length 8 may effectively 'over-correct' for the sequence bias caused by the motif's presence.

## Control for GC content
If you don't care for controlling for k-mer frequencies, you should at least control for GC content. In this case, call MEIRLOP with the options `--kmer 0 --gc`.

## Scoring sequences
MEIRLOP's results are most interpretable when sequence scores are symmetrical, ideally when they are normally distributed.

### Quantifying histone ChIP-seq signal
When calculating scores for your sequences, you should consider the scale of the genomic signal that score is capturing. For example, histone ChIP-seq is a wider-range signal (i.e. broad peaks), so you should quantify it +/- 500bp of the center of the genomic region from which you are drawing sequence.

Similar concerns govern quantification of experimentally derived covariates.

## Extracting sequences
A common complaint when using MEIRLOP was that creating a scored FASTA file was too much of a hassle. We implemented the `--bed` and `--fi` options to allow MEIRLOP to generate scored sequence data from a bed file and reference fasta file, respectively. However, this feature relies on the behavior of [`bedtools getfasta`](https://bedtools.readthedocs.io/en/latest/content/tools/getfasta.html), which has had undocumented changes as of version 2.29.2. Because of this, we do not recommend using these options in MEIRLOP at this time. If you truly must use these options, verify that your bedtools version is 2.29.0 at most.

For examples of how we made our scored fasta files, see our walkthrough, which details both [sequence extraction](walkthrough.ipynb#Get-sequence-of-DNase-I-Hypersensitive-sites) and [output](walkthrough.ipynb#Write-the-scored-FASTA-files).

## Length of sequences
In general, the length of your sequence usually corresponds to the length of the genomic peak for the signal you are quantifying. We have used MEIRLOP with sequences extracted from as small as +/- 75bp around the peak center, to as large as +/- 500bp around the peak center.

## Non-uniform sequence lengths
MEIRLOP expects that all sequences be the same length. Most motif enrichment analysis software does. But if you _really_ must analyze sequences of non-uniform length for some reason, you can call MEIRLOP with the `--length` option to control for sequence length as a covariate.



# How MEIRLOP works
This section is adapted from the MEIRLOP manuscript.

MEIRLOP is based on a logistic regression model for motif enrichment. At minimum, it accepts a list of scored sequences (in the [AME](http://meme-suite.org/doc/ame.html?man_type=web)-style scored FASTA format, with sequence headers consisting of a name and score separated by a space), and a motif database (in JASPAR format). MEIRLOP is executed in three parts: The scanning of sequences for motifs; the reduction of k-mer frequency (and optionally, other) covariates into a set of linearly uncorrelated principal components (AKA reduced covariates); and the logistic regression of motif presence against sequence scores and reduced covariates.

## Scanning sequences for motifs
To detect transcription factor binding motifs in genomic sequence, we use the [MOODS motif scanner](https://github.com/jhkorhonen/MOODS) to scan for sequence matching PWMs from the input motif set. The MOODS motif scanner internally takes two parameters to determine if a subsequence matches a motif matrix: a pseudocount and a p-value, which default to 0.001. MEIRLOP sets these parameters using arguments ‘--pcount’ and ‘--pval’.

## Principal component reduction of covariates
Although it is possible to directly input sequence-derived covariates into a logistic regression model, when using the k-mer frequencies of sequences as covariates, the multicollinearity of these frequencies (due to e.g. CpG islands) can lead to model instability and inaccurate parameter estimation. To account for this while preserving the ability to control for k-mer frequencies, we reduce multiple k-mer frequency covariates into a lower-dimensional set of principal components, converting the multicollinear predictors into a set of linearly uncorrelated predictors explaining 99% of the variance. We use the PCA implementation available in [scikit-learn](scikit-learn.org/). 

## Logistic regression for motif enrichment
We refer to a single resulting reduced covariates as $ x_c $.
Let p be the probability of a sequence containing a given motif $ m $, and let $ x_s $ be the score assigned to the sequence. We then model the log-odds of a sequence with score $ x_s $ containing motif $ m $ as:  
$ \log{\frac{p}{1-p}} = \beta_0 + \beta_s x_s + \sum_{c}^{n} \beta_c x_c $  
Where $ x_c $ refers to one of the $ n $ reduced covariates previous described, with $\beta_c$ being the corresponding coefficient. After maximum likelihood estimation of the coefficients and bias term, $ \beta_s $ can be interpreted as the change in the log-odds of a sequence containing motif $ m $, for a one unit increase in $ x_s $. To ensure that the maximum likelihood estimation converges, all predictor variables are standardized. The significance of the coefficient $ \beta_s $ is determined using the Wald test, as per the [Statsmodels](statsmodels.org/) implementation of logistic regression. To control for multiple hypothesis testing across multiple motifs, we applied Benjamini-Hochberg correction to the Wald test p-values.
This approach to motif enrichment allows controlling for sequence derived covariates such as GC content and k-mer frequencies, allowing the model to compensate for sequence bias similarly to certain set-based methods such as [HOMER](http://homer.ucsd.edu/homer/introduction/basics.html). 